<a href="https://colab.research.google.com/github/entanglesoftware/ai-learning/blob/main/lamaindex/lamaindex_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install deeplake==3.9.26
!pip install httpx==0.27.2
!pip install --upgrade openai llama-index
!pip install llama-index-vector-stores-docarray

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 351.8/351.8 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.7/431.7 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the 

In [ ]:
!pip install -q llama-index llama-index-vector-stores-chroma llama-index-vector-stores-deeplake openai tiktoken==0.6.0 chromadb==0.4.22 wikipedia deeplake langchain deeplake langchain-deeplake openai tiktoken
!pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.4/567.4 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.12.0
    Uninstalling openai-1.12.0:
      Successfully uninstalled openai-1.12.0


In [4]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["ACTIVELOOP_TOKEN"] = userdata.get('ACTIVELOOP_TOKEN')

In [5]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [15]:
from llama_index.core import download_loader

WikipediaReader = download_loader("WikipediaReader")

loader = WikipediaReader()
documents = loader.load_data(pages=['DeepSeek', 'Mistral_AI', 'Llama_(language_model)', 'Damian_Chong_Qui'], auto_suggest=False)

print(len(documents))

<ipython-input-15-2577dcc3acda>:3: DeprecationWarning: Call to deprecated function (or staticmethod) download_loader. (`download_loader()` is deprecated. Please install tool using pip install directly instead.)
  WikipediaReader = download_loader("WikipediaReader")


4


In [16]:
from llama_index.core.node_parser import SimpleNodeParser

# initializes the parser
parser = SimpleNodeParser.from_defaults(chunk_size = 512, chunk_overlap=20)

# Parse documents into nodes
nodes = parser.get_nodes_from_documents(documents)
print(len(nodes))

26


In [17]:
import deeplake
from llama_index.vector_stores.deeplake import DeepLakeVectorStore

my_activeloop_org_id = "entanglesoftware"
my_activeloop_dataset_name = "LlamaIndex_Vector"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

deeplake.delete(dataset_path, force=True);

# Create an index over the documents
vector_store = DeepLakeVectorStore(dataset_path=dataset_path)

print("Your Deep Lake dataset has been successfully created!")

Your Deep Lake dataset has been successfully created!


Your Deep Lake dataset has been successfully created!


In [18]:
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.core import load_index_from_storage

storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, show_progress=True
)

index.storage_context.persist()

print(index.summary)
print(f"Indexed {len(index.docstore.docs)} documents")

index = load_index_from_storage(index.storage_context)
print(index.summary)

Parsing nodes:   0%|          | 0/4 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/14 [00:00<?, ?it/s]

Uploading data to deeplake dataset.


100%|██████████| 14/14 [00:00<00:00, 38.68it/s]
/

Dataset(path='hub://entanglesoftware/LlamaIndex_Vector', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
   text       text      (14, 1)      str     None   
 metadata     json      (14, 1)      str     None   
 embedding  embedding  (14, 1536)  float32   None   
    id        text      (14, 1)      str     None   
None
Indexed 0 documents
None


In [19]:
from llama_index.core import GPTVectorStoreIndex

index = GPTVectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

response = query_engine.query("Who is the first player to score a triple-double in the East Asia Super League?")

print(response.response)

Damian Chong Qui is the first player to score a triple-double in the East Asia Super League.
